# Modeling School Budgets
- 2017-12-12
- William Surles
- This is for the Driven Data [school budget competition](https://www.drivendata.org/competitions/46/box-plots-for-education-reboot/) (its just for learning)
- I am using [this](https://github.com/datacamp/course-resources-ml-with-experts-budgets/blob/master/notebooks/1.0-full-model.ipynb) notebook provided by datacamp as a guide

In [1]:
%matplotlib inline
# from _future_ import division
# from _future_ import print_function

# ignore deprecation warnings in sklearn
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
import sys

# add the src directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(src_dir)

from data.multilabel import multilabel_sample_dataframe, multilabel_train_test_split
from features.SparseInteractions import SparseInteractions
from models.metrics import multi_multi_log_loss

# Load Data

In [2]:
file = 'data/TrainingData.csv'
df = pd.read_csv(file, index_col = 0)
print(df.shape)

(400277, 25)


# Resample Data

- Okay, so 400k rows might be a bit too many to work on locally, especially if I'm going to do bi-grams and interactions. I'll be here all day. 
- We'll sample down to 10k so that we can run the analysis faster and see how the models perform. 
- We will also create dummy variable for our labels and split our sampled dataset into training and test sets

In [3]:
LABELS = [
    'Function',
    'Object_Type', 
    'Operating_Status',
    'Position_Type',
    'Pre_K',
    'Reporting',
    'Sharing',
    'Student_Type',
    'Use']

NON_LABELS = [c for c in df.columns if c not in LABELS]

SAMPLE_SIZE = 20000

sampling = multilabel_sample_dataframe(
    df, 
    pd.get_dummies(df[LABELS]),
    size = SAMPLE_SIZE,
    min_count = 25, 
    seed = 43)

sampling.head()

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,...,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
227,Enrichment,NO_LABEL,School Reported,School,NO_LABEL,Club Advisor/Coach,Other Compensation/Stipend,NO_LABEL,PreK-12 Operating,Extracurricular Supp,...,NaN,NaN,NaN,NaN,NaN,NaN,1532.741660,NaN,NaN,NaN
1361,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Base Salary/Compensation,NO_LABEL,PreK-12 Operating,SALARIES OF PART TIME EMPLOYEE,...,NaN,NaN,0.00431,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,109.667938,GENERAL ELEMENTARY EDUCATION,NaN,REGULAR INSTRUCTION
1517,Professional Development,ISPD,School on Central Budgets,Non-School,Gifted,Instructional Coach,Base Salary/Compensation,Non PreK,PreK-12 Operating,Personal Services - Teachers,...,NaN,District Wide Resources,0.00000,NaN,GIFTED AND TALENTED,GIFTED & TALENTED COACH,1459.330000,Instruction - Regular,General Purpose School,NaN
1524,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,Non-Operating,Supplies and Materials,...,Supplies And Materials,NaN,NaN,Other Community Services *,NaN,NaN,2304.430000,NaN,Title I - Disadvantaged Children/Targeted Assi...,TITLE I PI+HOMELESS
1545,School Administration,Leadership,School Reported,School,Unspecified,Sec/Clerk/Other Admin,Other Compensation/Stipend,NO_LABEL,PreK-12 Operating,ADDITIONAL/EXTRA DUTY PAY/STIP,...,NaN,NaN,NaN,NON-PROJECT,NaN,OFFICE/ADMINISTRATIVE SUPPORT,996.870000,SUPPORT SERVICES-SCHOOL ADMIN,NaN,SCHOOL ADMINISTRATION


In [4]:
dummy_labels = pd.get_dummies(sampling[LABELS])
dummy_labels.head()

,Function_Aides Compensation,Function_Career & Academic Counseling,Function_Communications,Function_Curriculum Development,Function_Data Processing & Information Services,Function_Development & Fundraising,Function_Enrichment,Function_Extended Time & Tutoring,Function_Facilities & Maintenance,Function_Facilities Planning,...,Student_Type_Special Education,Student_Type_Unspecified,Use_Business Services,Use_ISPD,Use_Instruction,Use_Leadership,Use_NO_LABEL,Use_O&M,Use_Pupil Services & Enrichment,Use_Untracked Budget Set-Aside
227,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1361,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1517,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1524,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1545,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [5]:
X_train, X_test, y_train, y_test = multilabel_train_test_split(
    sampling[NON_LABELS],
    dummy_labels,
    size = 0.2,
    min_count = 25, 
    seed = 43)

In [6]:
X_train.head()

,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
227,Extracurricular Supp,NaN,NaN,EXTRA CURRICULAR SP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1532.741660,NaN,NaN,NaN
1361,SALARIES OF PART TIME EMPLOYEE,NaN,GENERAL FUND,"Teacher, Short Term Sub",Regular,NaN,NaN,NaN,0.00431,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,109.667938,GENERAL ELEMENTARY EDUCATION,NaN,REGULAR INSTRUCTION
1524,Supplies and Materials,NaN,Community Services,NaN,NaN,NaN,Supplies And Materials,NaN,NaN,Other Community Services *,NaN,NaN,2304.430000,NaN,Title I - Disadvantaged Children/Targeted Assi...,TITLE I PI+HOMELESS
1545,ADDITIONAL/EXTRA DUTY PAY/STIP,NaN,GENERAL FUND,Secretary I,Regular,NaN,NaN,NaN,NaN,NON-PROJECT,NaN,OFFICE/ADMINISTRATIVE SUPPORT,996.870000,SUPPORT SERVICES-SCHOOL ADMIN,NaN,SCHOOL ADMINISTRATION
1604,Personal Services - Teachers,NaN,NaN,NaN,NaN,Regular Instruction,NaN,NaN,0.20000,NaN,NaN,TEACHER,1297.620000,Instruction - Regular,General Purpose School,NaN


In [7]:
y_train.head()

,Function_Aides Compensation,Function_Career & Academic Counseling,Function_Communications,Function_Curriculum Development,Function_Data Processing & Information Services,Function_Development & Fundraising,Function_Enrichment,Function_Extended Time & Tutoring,Function_Facilities & Maintenance,Function_Facilities Planning,...,Student_Type_Special Education,Student_Type_Unspecified,Use_Business Services,Use_ISPD,Use_Instruction,Use_Leadership,Use_NO_LABEL,Use_O&M,Use_Pupil Services & Enrichment,Use_Untracked Budget Set-Aside
227,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1361,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1524,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1545,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
1604,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0


# Create preprocessing tools

- We need to make some pre-processing tools for out text and numeric data.
- The `combine_text_columns` function will take a DataFrame of text columns and return a single series where all of the text in the columns has been joined together.

In [8]:
NUMERIC_COLUMNS = ['FTE', 'Total']

def combine_text_columns(data_frame, to_drop=NUMERIC_COLUMNS + LABELS):
    """ converts all text in each row of data_frame to single vector 
        :param data_frame: The data as read in with read_csv (no preprocessing necessary)
        :param to_drop: (optional): Removes the numeric and label columns be default
    """
    
    # Drop non-text columns that are in the df
    to_drop = set(to_drop) & set(data_frame.columns.tolist())
    text_data = data_frame.drop(to_drop, axis = 1)
    
    # Replace nans with blanks
    text_data.fillna("", inplace=True)
    
    # Join all text items in a row with a space in between
    return text_data.apply(lambda x: " ".join(x), axis=1)

- We also need to create `FunctionTransformer` objects that select our text and numeric data from the dataframe

In [9]:
from sklearn.preprocessing import FunctionTransformer

get_text_data = FunctionTransformer(combine_text_columns, validate = False)
get_numeric_data = FunctionTransformer(lambda x: x[NUMERIC_COLUMNS], validate = False)

In [10]:
get_text_data.fit_transform(sampling.head(5))

227     Extracurricular Supp   EXTRA CURRICULAR SP    ...
1361    SALARIES OF PART TIME EMPLOYEE  GENERAL FUND T...
1517    Personal Services - Teachers   GIFTED AND TALE...
1524    Supplies and Materials  Community Services    ...
1545    ADDITIONAL/EXTRA DUTY PAY/STIP  GENERAL FUND S...
dtype: object

In [11]:
get_numeric_data.fit_transform(sampling.head(5))

,FTE,Total
227,NaN,1532.741660
1361,0.00431,109.667938
1517,0.00000,1459.330000
1524,NaN,2304.430000
1545,NaN,996.870000


- Finally, we create a custom scoring method that uses the `multi_multi_log_loss` function that is the evaluation metric for the competition

In [12]:
from sklearn.metrics.scorer import make_scorer

log_loss_scorer = make_scorer(multi_multi_log_loss)

# Train model pipeline

Now we'll train the final pipeline from the course. It will:
- take text and numeric data
- do necessary preprocessing
- train the classifier

Pretty simple right?

In [13]:
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import Imputer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MaxAbsScaler

TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

In [14]:
%%time

## set a reasonable number of features before adding interactions
chi_k = 300

## create the pipeline object
pl = Pipeline([
    ('union', FeatureUnion(
        transformer_list = [
            ('numeric_features', Pipeline([
                ('selector', get_numeric_data),
                ('imputer', Imputer())
            ])),
            ('text_features', Pipeline([
                ('selector', get_text_data),
                ('vectorizer', HashingVectorizer(
                    token_pattern = TOKENS_ALPHANUMERIC,
                    non_negative = True,
                    norm = None,
                    binary = False,
                    ngram_range = (1, 2))),
                ('dim_red', SelectKBest(chi2, chi_k))
            ]))
        ])),
    ('int', SparseInteractions(degree = 2)),
    ('scale', MaxAbsScaler()),
    ('clf', OneVsRestClassifier(RandomForestClassifier()))
])

## fit the pipeline to our training data
pl.fit(X_train, y_train.values)

## print the score
print("Logloss score on test data: ", log_loss_scorer(pl, X_test, y_test.values))

/Users/williamsurles/anaconda3/envs/python_datacamp/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/Users/williamsurles/anaconda3/envs/python_datacamp/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/Users/williamsurles/anaconda3/envs/python_datacamp/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


Logloss score on test data:  10.716614787
CPU times: user 4min 3s, sys: 1.56 s, total: 4min 4s
Wall time: 4min 5s


# Predict holdout set and write submission

- Finally, we want to use our trianed pipeline to predict the holdout dataset. 
- We will write our predictions to a file, `predictions.csv`, that we can summit on DrivenData.

In [15]:
## Load holdout data
file = 'data/TestData.csv'
holdout = pd.read_csv(file, index_col = 0)
holdout.head()

,Object_Description,Program_Description,SubFund_Description,Job_Title_Description,Facility_or_Department,Sub_Object_Description,Location_Description,FTE,Function_Description,Position_Extra,Text_4,Total,Text_2,Text_3,Fund_Description,Text_1
180042,Student Meals/Room/Other,Basic Educational Services,NaN,NaN,NaN,Line Item that is paid with Campus' money,School,NaN,Instruction,NaN,NaN,3999.910000,NaN,NaN,General Fund,NaN
28872,Extra Duty/Signing Bonus Pay,Undistributed,NaN,CHEERLEADER DIR,NaN,General,School,NaN,Cocurricular & Extra Curricular Activities,NaN,NaN,3447.320213,NaN,NaN,General Fund,NaN
186915,Professional Salaries,Bilingual Education,NaN,T-EL 1ST BIL,NaN,General,School,1.0,Instruction,NaN,NaN,52738.780869,NaN,NaN,General Fund,NaN
412396,Professional Salaries,Bilingual Education,NaN,T-EL 2ND BIL,NaN,General,School,1.0,Instruction,NaN,NaN,69729.263191,NaN,NaN,General Fund,NaN
427740,Salaries for Support Personnel,Undistributed,NaN,CLERK III- SCH,NaN,General,School,1.0,School Leadership,NaN,NaN,29492.834215,NaN,NaN,General Fund,NaN


In [16]:
holdout.shape

(50064, 16)

In [18]:
## Make predictions
predictions_prob = pl.predict_proba(holdout)
predictions_bool = pl.predict(holdout)

/Users/williamsurles/anaconda3/envs/python_datacamp/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/Users/williamsurles/anaconda3/envs/python_datacamp/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [19]:
print(len(predictions_prob))
print(len(predictions_bool))

50064
50064


In [20]:
print(len(predictions_prob[0]))
print(len(predictions_bool[0]))

104
104


In [21]:
type(predictions_prob)

numpy.ndarray

In [ ]:
predictions_prob_trans = list(map(list, zip(*predictions_prob)))

In [ ]:
print(len(predictions_prob_trans))
print(len(predictions_prob_trans[0]))

In [22]:
predictions_prob[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.1325    ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.3052381 ,  0.        ,  0.        ,  0.        ,
        0.23107143,  0.        ,  0.205     ,  0.        ,  0.17333333,
        0.        ,  0.        ,  0.        ,  0.        ,  0.14829365,
        0.        ,  0.        ,  0.58583333,  0.        ,  0.        ,
        0.        ,  0.21916667,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.32857143,  0.        ,  0.        ,  0.        ,  0.30928571,
        0.        ,  0.15357143,  0.        ,  0.        ,  0.  

In [ ]:
## Format the prediction df for submission
prediction_df = pd.DataFrame(
    columns = pd.get_dummies(df[LABELS], prefix_sep = '__').columns,
    index = holdout.index,
    data = predictions_prob_trans)

prediction_df.head()

In [ ]:
## Save prediction_df to csv
prediction_df.to_csv("predictions.csv")